<a href="https://colab.research.google.com/github/kazu1999/LENet/blob/main/DLModel_map_lidar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

mapとlidar情報を組み込んだ予測を行う

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip drive/MyDrive/lidar_map_emb/for_drive.zip #ダウンロードしたzipファイルのパスをいれる

In [ ]:
!pip install folium
!pip install pytorch-summary
!pip install numpy-quaternion

In [ ]:
import pandas as pd
import folium
import pickle
import numpy as np
from sklearn.neighbors import KDTree
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import statistics
import math
from sklearn.metrics import confusion_matrix
import seaborn as sns

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import os
from torchsummary import summary
import glob
import shutil


%matplotlib inline

# データ結合

In [ ]:
import pandas as pd
import glob
import numpy as np
import quaternion
import math
import os

def cal_lat_lon(gt_p_x, gt_p_y, loc_p_x, loc_p_y, gt_qua_x, gt_qua_y, gt_qua_z, gt_qua_w):
  error = ((gt_p_x - loc_p_x)**2 + (gt_p_y - loc_p_y)**2)**0.5
  quat = np.quaternion(gt_qua_w, gt_qua_x, gt_qua_y, gt_qua_z)
  deg_ori =  math.degrees(quaternion.as_rotation_vector(quat)[2])
  if loc_p_x == gt_p_x:
    loc_p_x = loc_p_x + 0.001
  deg_error = math.degrees(math.atan((loc_p_y - gt_p_y)/(loc_p_x - gt_p_x)))
  deg = abs(deg_ori - deg_error)
  lon = error*math.cos(math.radians(deg))
  lat = error*math.sin(math.radians(deg))
  return lat, lon

def error(x):
  return ((x['gt_x']-x[' out_p_x'])**2 + (x['gt_y']-x[' out_p_y'])**2)**0.5

def lon(x):
  lat, lon = cal_lat_lon(x['gt_position_x'],x['gt_position_y'],x['loc_position_x'],x['loc_position_y'],x['gt_orientation_x'],x['gt_orientation_y'],x['gt_orientation_z'],x['gt_orientation_w'])
  return abs(lon)

def lat(x):
  lat, lon = cal_lat_lon(x['gt_position_x'],x['gt_position_y'],x['loc_position_x'],x['loc_position_y'],x['gt_orientation_x'],x['gt_orientation_y'],x['gt_orientation_z'],x['gt_orientation_w'])
  return abs(lat)


path_num_list=[1,2,3,4,5,6,7]

for path_num in path_num_list:
    glob_path = 'for_drive/path' + str(path_num) + '/*'
    dir_list = glob.glob(glob_path)
    print(path_num)
    print(dir_list)
    print()
    for direc in dir_list:
      df = pd.read_csv(direc + '/merged.csv')

      df['lon'] = df.apply(lon,axis=1)
      df['lat'] = df.apply(lat,axis=1)
      print(df)

      df.to_csv(direc + '/merged2.csv')

# データ分析

In [ ]:
path_num_list=[1,2,3,4,5,6,7]

merged_merged = pd.DataFrame()

for path_num in path_num_list:
    glob_path = 'for_drive/path' + str(path_num) + '/*'
    dir_list = glob.glob(glob_path)
    print(path_num)

    for dirr in dir_list:
      print(dirr)
      spl_dirr=dirr.split('/')
      rosbag_name = spl_dirr[2]

      merged = pd.read_csv(dirr+'/merged2.csv')
      merged['rosbag']=rosbag_name
      merged['path']=path_num
      merged_merged = pd.concat([merged_merged,merged])



In [ ]:
merged_merged

In [ ]:
merged_merged.columns

In [ ]:
merged_out = merged_merged.loc[:,['gt_position_x','gt_position_y','error','lon','lat','path']]

In [ ]:
merged_out.to_csv('vis.csv')

## 各pathのerror可視化

### path1

In [ ]:
path1_merged = merged_merged[merged_merged['path']==1]
path1_merged = path1_merged.reset_index(drop=True)

In [ ]:
path1_merged.to_csv('path1_merged.csv')

In [ ]:
path1_merged

In [ ]:
plt.plot(path1_merged['error'])
plt.hlines(path1_merged['error'].mean(), 0, len(path1_merged))
plt.show()

print('avg : ')
print(path1_merged['error'].mean())

In [ ]:
plt.plot(path1_merged['lon'])
plt.hlines(path1_merged['lon'].mean(), 0, len(path1_merged))
plt.show()

print('avg : ')
print(path1_merged['lon'].mean())

In [ ]:
plt.plot(path1_merged['lat'])
plt.hlines(path1_merged['lat'].mean(), 0, len(path1_merged))
plt.show()

print('avg : ')
print(path1_merged['lat'].mean())

### path2

In [ ]:
path2_merged = merged_merged[merged_merged['path']==2]
path2_merged = path2_merged.reset_index(drop=True)

In [ ]:
path2_merged

In [ ]:
path2_merged.to_csv('path2_merged.csv')

In [ ]:
plt.plot(path2_merged['error'])
plt.hlines(path2_merged['error'].mean(), 0, len(path2_merged))
plt.show()

print('avg : ')
print(path2_merged['error'].mean())

In [ ]:
plt.plot(path2_merged['lon'])
plt.hlines(path2_merged['lon'].mean(), 0, len(path2_merged))
plt.show()

print('avg : ')
print(path2_merged['lon'].mean())

In [ ]:
plt.plot(path2_merged['lat'])
plt.hlines(path2_merged['lat'].mean(), 0, len(path2_merged))
plt.show()

print('avg : ')
print(path2_merged['lat'].mean())

### path3

In [ ]:
path3_merged = merged_merged[merged_merged['path']==3]
path3_merged = path3_merged.reset_index(drop=True)

In [ ]:
path3_merged

In [ ]:
path3_merged.to_csv('path3_merged.csv')

In [ ]:
plt.plot(path3_merged['error'])
plt.hlines(path3_merged['error'].mean(), 0, len(path3_merged))
plt.show()

print('avg : ')
print(path3_merged['error'].mean())

In [ ]:
plt.plot(path3_merged['lon'])
plt.hlines(path3_merged['lon'].mean(), 0, len(path3_merged))
plt.show()

print('avg : ')
print(path3_merged['lon'].mean())

In [ ]:
plt.plot(path3_merged['lat'])
plt.hlines(path3_merged['lat'].mean(), 0, len(path3_merged))
plt.show()

print('avg : ')
print(path3_merged['lat'].mean())

### path4

In [ ]:
path4_merged = merged_merged[merged_merged['path']==4]
path4_merged = path4_merged.reset_index(drop=True)

In [ ]:
path4_merged

In [ ]:
path4_merged.to_csv('path4_merged.csv')

In [ ]:
plt.plot(path4_merged['error'])
plt.hlines(path4_merged['error'].mean(), 0, len(path4_merged))
plt.show()

print('avg : ')
print(path4_merged['error'].mean())

In [ ]:
plt.plot(path4_merged['lon'])
plt.hlines(path4_merged['lon'].mean(), 0, len(path4_merged))
plt.show()

print('avg : ')
print(path4_merged['lon'].mean())

In [ ]:
plt.plot(path4_merged['lat'])
plt.hlines(path4_merged['lat'].mean(), 0, len(path4_merged))
plt.show()

print('avg : ')
print(path4_merged['lat'].mean())

### path5

In [ ]:
path5_merged = merged_merged[merged_merged['path']==5]
path5_merged = path5_merged.reset_index(drop=True)

In [ ]:
path5_merged

In [ ]:
path5_merged.to_csv('path5_merged.csv')

In [ ]:
plt.plot(path5_merged['error'])
plt.hlines(path5_merged['error'].mean(), 0, len(path5_merged))
plt.show()

print('avg : ')
print(path5_merged['error'].mean())

In [ ]:
plt.plot(path5_merged['lon'])
plt.hlines(path5_merged['lon'].mean(), 0, len(path5_merged))
plt.show()

print('avg : ')
print(path5_merged['lon'].mean())

In [ ]:
plt.plot(path5_merged['lat'])
plt.hlines(path5_merged['lat'].mean(), 0, len(path5_merged))
plt.show()

print('avg : ')
print(path5_merged['lat'].mean())

### path6

In [ ]:
path6_merged = merged_merged[merged_merged['path']==6]
path6_merged = path6_merged.reset_index(drop=True)

In [ ]:
path6_merged

In [ ]:
path6_merged.to_csv('path6_merged.csv')

In [ ]:
plt.plot(path6_merged['error'])
plt.hlines(path6_merged['error'].mean(), 0, len(path6_merged))
plt.show()

print('avg : ')
print(path6_merged['error'].mean())

In [ ]:
plt.plot(path6_merged['lon'])
plt.hlines(path6_merged['lon'].mean(), 0, len(path6_merged))
plt.show()

print('avg : ')
print(path6_merged['lon'].mean())

In [ ]:
plt.plot(path6_merged['lat'])
plt.hlines(path6_merged['lat'].mean(), 0, len(path6_merged))
plt.show()

print('avg : ')
print(path6_merged['lat'].mean())

### path7

In [ ]:
path7_merged = merged_merged[merged_merged['path']==7]
path7_merged = path7_merged.reset_index(drop=True)

In [ ]:
path7_merged

In [ ]:
path7_merged.to_csv('path7_merged.csv')

In [ ]:
plt.plot(path7_merged['error'])
plt.hlines(path7_merged['error'].mean(), 0, len(path7_merged))
plt.show()

print('avg : ')
print(path7_merged['error'].mean())

In [ ]:
plt.plot(path7_merged['lon'])
plt.hlines(path7_merged['lon'].mean(), 0, len(path7_merged))
plt.show()

print('avg : ')
print(path7_merged['lon'].mean())

In [ ]:
plt.plot(path7_merged['lat'])
plt.hlines(path7_merged['lat'].mean(), 0, len(path7_merged))
plt.show()

print('avg : ')
print(path7_merged['lat'].mean())

# 前処理

# ダウンサンプリングレート選択

In [ ]:
# '' or _0.25, _0.5, _0.75, _1
lidar_ds_rate = '_1'
map_ds_rate = '_1'


## test train data 分割

In [ ]:
merged_merged.to_csv('merged_merged.csv')

In [ ]:
merged_merged.columns

In [ ]:
def pcd_path(x):
  return 'path'+str(x['path'])+'/'+ x['rosbag']+'/'+'embedded'+lidar_ds_rate+'/'+str(x['pcd_filename'])+'_cloud.bin'

merged_merged['pcd_path'] = merged_merged.apply(pcd_path,axis=1)


In [ ]:
def map_path(x):
  return 'path'+str(x['path'])+'/'+ x['rosbag']+'/'+'map_emb'+map_ds_rate+'/'+str(x['pcd_filename'])+'_cloud.bin'

merged_merged['map_path'] = merged_merged.apply(map_path,axis=1)

In [ ]:
merged_merged

### 各pathで分割

#### path1

In [ ]:
# path1

path1_extract_merged =merged_merged[merged_merged['path']==1]
serial_num = pd.RangeIndex(start=1, stop=len(path1_extract_merged.index) + 1, step=1)
path1_extract_merged['no'] = serial_num

def istest(x):
  if x['no']< len(path1_extract_merged)*0.1:
    return True
  return False

path1_extract_merged['istest'] = path1_extract_merged.apply(istest,axis=1)


#### path2

In [ ]:
# path2

path2_extract_merged =merged_merged[merged_merged['path']==2]
serial_num = pd.RangeIndex(start=1, stop=len(path2_extract_merged.index) + 1, step=1)
path2_extract_merged['no'] = serial_num

def istest(x):
  if x['no']< len(path2_extract_merged)*0.1:
    return True
  return False

path2_extract_merged['istest'] = path2_extract_merged.apply(istest,axis=1)

#### path3

In [ ]:
# path3

path3_extract_merged =merged_merged[merged_merged['path']==3]
serial_num = pd.RangeIndex(start=1, stop=len(path3_extract_merged.index) + 1, step=1)
path3_extract_merged['no'] = serial_num

def istest(x):
  if x['no']< len(path3_extract_merged)*0.1:
    return True
  return False

path3_extract_merged['istest'] = path3_extract_merged.apply(istest,axis=1)

#### path4

In [ ]:
# path4

path4_extract_merged =merged_merged[merged_merged['path']==4]
serial_num = pd.RangeIndex(start=1, stop=len(path4_extract_merged.index) + 1, step=1)
path4_extract_merged['no'] = serial_num

def istest(x):
  if x['no']< len(path4_extract_merged)*0.1:
    return True
  return False

path4_extract_merged['istest'] = path4_extract_merged.apply(istest,axis=1)

#### path5

In [ ]:
# path5

path5_extract_merged =merged_merged[merged_merged['path']==5]
serial_num = pd.RangeIndex(start=1, stop=len(path5_extract_merged.index) + 1, step=1)
path5_extract_merged['no'] = serial_num

def istest(x):
  if x['no']< len(path5_extract_merged)*0.1:
    return True
  return False

path5_extract_merged['istest'] = path5_extract_merged.apply(istest,axis=1)

#### path6

In [ ]:
# path6

path6_extract_merged =merged_merged[merged_merged['path']==6]
serial_num = pd.RangeIndex(start=1, stop=len(path6_extract_merged.index) + 1, step=1)
path6_extract_merged['no'] = serial_num

def istest(x):
  if x['no']< len(path6_extract_merged)*0.1:
    return True
  return False

path6_extract_merged['istest'] = path6_extract_merged.apply(istest,axis=1)

#### path7

In [ ]:
# path7

path7_extract_merged =merged_merged[merged_merged['path']==7]
serial_num = pd.RangeIndex(start=1, stop=len(path7_extract_merged.index) + 1, step=1)
path7_extract_merged['no'] = serial_num

def istest(x):
  if x['no']< len(path7_extract_merged)*0.1:
    return True
  return False

path7_extract_merged['istest'] = path7_extract_merged.apply(istest,axis=1)

### 各pathの分割をまとめる

In [ ]:
all_df = pd.DataFrame()
all_df = pd.concat([all_df,path1_extract_merged])
all_df = pd.concat([all_df,path2_extract_merged])
all_df = pd.concat([all_df,path3_extract_merged])
all_df = pd.concat([all_df,path4_extract_merged])
all_df = pd.concat([all_df,path5_extract_merged])
all_df = pd.concat([all_df,path6_extract_merged])
all_df = pd.concat([all_df,path7_extract_merged])

serial_num = pd.RangeIndex(start=1, stop=len(all_df.index) + 1, step=1)
all_df['id'] = serial_num

In [ ]:
all_df.columns

In [ ]:
all_df

In [ ]:
all_extract_df = all_df.loc[:,['id','pcd_path','map_path','istest','error','lon','lat','rosbag','path']]

In [ ]:
test_df = all_extract_df[all_extract_df['istest']==True]
train_df = all_extract_df[all_extract_df['istest']==False]

In [ ]:
test_df

In [ ]:
train_df

## 特徴抽出ベクトルの処理

In [ ]:
# embがないファイルの削除
test_vector=[]
train_vector=[]
test_pcd_unavai_id_list=[]
train_pcd_unavai_id_list=[]

for index, row in test_df.iterrows():
  if os.path.exists('for_drive/'+row['pcd_path']) and os.path.exists('for_drive/'+row['map_path']) :
    lidar_vector = np.fromfile('for_drive/'+row['pcd_path'],dtype=np.float32)
    map_vector = np.fromfile('for_drive/'+row['map_path'],dtype=np.float32)
    vector = np.concatenate([lidar_vector, map_vector], 0)
    test_vector.append(vector)
  else:
    test_pcd_unavai_id_list.append(row['id'])

for index, row in train_df.iterrows():
  if os.path.exists('for_drive/'+row['pcd_path']) and os.path.exists('for_drive/'+row['map_path']):
    lidar_vector = np.fromfile('for_drive/'+row['pcd_path'],dtype=np.float32)
    map_vector = np.fromfile('for_drive/'+row['map_path'],dtype=np.float32)
    vector = np.concatenate([lidar_vector, map_vector], 0)
    train_vector.append(vector)
  else:
    train_pcd_unavai_id_list.append(row['id'])


In [ ]:
print(test_pcd_unavai_id_list)
print(train_pcd_unavai_id_list)

In [ ]:
test_df = test_df[~test_df['id'].isin(test_pcd_unavai_id_list)]

In [ ]:
test_df

In [ ]:
train_df = train_df[~train_df['id'].isin(train_pcd_unavai_id_list)]

In [ ]:
train_df

# モデル実装

In [ ]:
# 目的変数の選択 error or lon or lat
target = 'error'

In [ ]:
#ML
# testdata traindata に分ける

X_train = train_vector
y_train = train_df[target].to_list()
X_test = test_vector
y_test = test_df[target].to_list()


X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

X_train = X_train.reshape([len(X_train),1,512])
X_test = X_test.reshape([len(X_test),1,512])

#y_train = y_train.reshape([len(y_train),1])
#y_test = y_test.reshape([len(y_test),1])

In [ ]:
y_test = y_test.astype('float32')
y_train = y_train.astype('float32')

In [ ]:
# Dataset 作成
# 参考 https://qiita.com/mathlive/items/2a512831878b8018db02
class TrainDataSets(torch.utils.data.Dataset):
  def __init__(self, transform = None):
    self.transform = transform

    self.data = X_train
    self.target = y_train

    self.datanum = len(X_train)

  def __len__(self):
    return self.datanum

  def __getitem__(self,idx):
    out_data = self.data[idx]
    out_target = self.target[idx]

    if self.transform:
      out_data = self.transform(out_data)

    return out_data, out_target

In [ ]:
# Dataset 作成
class TestDataSets(torch.utils.data.Dataset):
  def __init__(self, transform = None):
    self.transform = transform

    self.data = X_test
    self.target = y_test

    self.datanum = len(X_test)

  def __len__(self):
    return self.datanum

  def __getitem__(self,idx):
    out_data = self.data[idx]
    out_target = self.target[idx]

    if self.transform:
      out_data = self.transform(out_data)

    return out_data, out_target

In [ ]:
trans = torchvision.transforms.ToTensor()

In [ ]:
traindata = TrainDataSets(trans)
testdata = TestDataSets(trans)

batch_size = 64

train_dataloader = DataLoader(traindata, batch_size=batch_size)
test_dataloader = DataLoader(testdata, batch_size=batch_size)

In [ ]:
# model
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(512,1),
        nn.ReLU()
    )

  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch%1==0:
      loss, current = loss.item(), batch*len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model):
  size = len(dataloader.dataset)
  model.eval()
  test_loss, diff = 0,0
  with torch.no_grad():
    for X,y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred,y).item()
  test_loss /= size
  print(f"Avg loss: {test_loss:>8f} \n")
  return test_loss

In [ ]:
epoch = 5
losses=[]
for t in range(epoch):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  losses.append(test(test_dataloader, model))
print("Done!")

In [ ]:
plt.plot(losses)
plt.show()

In [ ]:
torch.save(model, target+lidar_ds_rate+map_ds_rate+'.pth')

In [ ]:
# 学習ずみモデルの読み込み．
model = torch.load('model/error_1_1.pth') # 学習ずみモデルのパスを入れる

In [ ]:
def exc(x):
  if x[target]>0.6:
    return 0
  return x[target]

test_df[target] = test_df.apply(exc,axis=1)

test_df = test_df.reset_index(drop=True)

In [ ]:
y_pred=[]

time_list = []
for X,y in test_dataloader:
  X, y = X.to(device), y.to(device)
  start_time = time.time()
  pred = model(X)
  end_time = time.time()
  dur_time = end_time - start_time
  time_list.append(dur_time)
  for i in range(len(pred)):
    y_pred.append(pred[i].item())
print(sum(time_list)/len(time_list))
print(len(time_list))

In [ ]:
print(y_pred)

In [ ]:
y_true = test_df[target].values.tolist()
print(y_true)

# 評価

## モデルの状態

In [ ]:
summary(model, (1,1,1,512))

# 可視化

In [ ]:
plt.plot(y_true)
plt.plot(y_pred)
plt.show()

## 決定係数

In [ ]:
# サンプルデータ
# R2(決定係数)
r2 = r2_score(y_true,                        # 実測値
              y_pred,                        # 予測値
              sample_weight=None,            # サンプルの重み
              multioutput="uniform_average"  # 複数スコアの集計方法
             )

print(r2)

## MSE

In [ ]:
from sklearn.metrics import mean_squared_error

# MSE算出
mse  = mean_squared_error(y_true,                         # 実測値
                          y_pred,                         # 予測値
                          sample_weight=None,             # サンプルの重み
                          multioutput='uniform_average',  # 複数スコアの集計方法
                          squared=True                    # True(MSEを算出), False(RMSEを算出)
                          )

print(mse)

## RMSE

In [ ]:
# RMSE算出
rmse  = mean_squared_error(y_true,                         # 実測値
                           y_pred,                         # 予測値
                           sample_weight=None,             # サンプルの重み
                           multioutput='uniform_average',  # 複数スコアの集計方法
                           squared=False                   # True(MSEを算出), False(RMSEを算出)
                          )

print(rmse)

## MAE

In [ ]:
# MAE算出
mae = mean_absolute_error(y_true,                       # 実測値
                          y_pred,                       # 予測値
                          sample_weight = None,         # サンプルの重み
                          multioutput='uniform_average' # 複数スコアの集計方法
                          )

print(mae)

In [ ]:
test_df['pred'] = y_pred

In [ ]:
test_df

In [ ]:
def exc(x):
  if x[target]>0.6:
    return 0
  return x[target]

test_df[target] = test_df.apply(exc,axis=1)

In [ ]:
plt.plot(test_df[target])
plt.plot(test_df['pred'])
plt.show()

In [ ]:
test_df.to_csv('result'+lidar_ds_rate+map_ds_rate+'_'+target+'.csv')

## 各path予測

### path1

In [ ]:
path1_test_df = test_df[test_df['path']==1]

In [ ]:
path1_test_df

In [ ]:



plt.plot(path1_test_df[target],label='actual')
plt.plot(path1_test_df['pred'],label='pred')
plt.legend()
plt.show()

### path2

In [ ]:
path2_test_df = test_df[test_df['path']==2]
error = path2_test_df[target]
pred = path2_test_df['pred']
error = error.reset_index(drop=True)
pred = pred.reset_index(drop=True)
plt.plot(error,label='actual')
plt.plot(pred,label='pred')
plt.legend()
plt.show()

### path3

In [ ]:
path3_test_df = test_df[test_df['path']==3]
error = path3_test_df[target]
pred = path3_test_df['pred']
error = error.reset_index(drop=True)
pred = pred.reset_index(drop=True)
plt.plot(error,label='actual')
plt.plot(pred,label='pred')
plt.legend()
plt.show()

### path4

In [ ]:
path4_test_df = test_df[test_df['path']==4]
error = path4_test_df[target]
pred = path4_test_df['pred']
error = error.reset_index(drop=True)
pred = pred.reset_index(drop=True)
plt.plot(error,label='actual')
plt.plot(pred,label='pred')
plt.legend()
plt.show()

### path5

In [ ]:
path5_test_df = test_df[test_df['path']==5]
error = path5_test_df[target]
pred = path5_test_df['pred']
error = error.reset_index(drop=True)
pred = pred.reset_index(drop=True)
plt.plot(error,label='actual')
plt.plot(pred,label='pred')
plt.legend()
plt.show()

### path6

In [ ]:
path6_test_df = test_df[test_df['path']==6]
error = path6_test_df[target]
pred = path6_test_df['pred']
error = error.reset_index(drop=True)
pred = pred.reset_index(drop=True)
plt.plot(error,label='actual')
plt.plot(pred,label='pred')
plt.legend()
plt.show()

### path7

In [ ]:
path7_test_df = test_df[test_df['path']==7]
error = path7_test_df[target]
pred = path7_test_df['pred']
error = error.reset_index(drop=True)
pred = pred.reset_index(drop=True)
plt.plot(error,label='actual')
plt.plot(pred,label='pred')
plt.legend()
plt.show()